## Infer the producers of a Wasm binary experiment

In [128]:
# Load those Wasm from the dataset with producers information
import json
import pandas as pd

#metadata = open("../RQ1/filtered-binaries-metadata_2/filtered.pretty.json", "r").read()
metadata = open("../RQ1/all-binaries-metadata/all.pretty.json", "r").read()

metadata = json.loads(metadata)

d = {
    'wasm': [],
    'producer': [],
    'language': [],
    'tool': [],
    'tool_version': []
}

for k in metadata:
    if 'producers' in metadata[k]:
        if metadata[k]['producers']:
            prod = metadata[k]['producers']
            if 'language' in prod:
                d['language'].append(list(prod['language'].keys())[0])
            else:
                d['language'].append(None)
                
            if 'processed-by' in prod:
                processor = prod['processed-by']
                # do a preprocessing here, to select the tool and the version only
                d['producer'].append(prod['processed-by'].__str__())
                tool = list(prod['processed-by'].keys())[0]
                #if len(processor) > 1:
                #    print(processor)
                    
                    
                d['tool'].append(tool)
                
                # getting version
                if tool == 'rustc':
                    # the version is the next 4-(nightly) characters
                    version = processor[tool][:8]
                    d['tool_version'].append(f"{tool} {version}")
                elif tool == 'clang':
                    
                    version = processor[tool][:6]
                    d['tool_version'].append(f"{tool} {version}")
                elif tool == 'Go cmd/compile':
                    version = processor[tool][6:18]
                    tool = 'Go'
                    
                    d['tool_version'].append(f"{tool} {version}")
                elif tool == 'Apple LLVM':
                    
                    version = processor[tool][:8]
                    d['tool_version'].append(f"clang {version}")
                else:
                    d['tool_version'].append(f"{tool} -")
                    
                
                
            else:
                d['producer'].append(None)
                d['tool'].append(None)
                d['tool_version'].append(None)
                
            d['wasm'].append(k)
            
            
d = pd.DataFrame(d)
display(d)

display(d.groupby('language').count().reset_index())

# 1723 programs....maybe not ebough for a correct inference ?

,wasm,producer,language,tool,tool_version
0,c8026b77b08a48698d98d8f08ad608e7527833b5fe3cf1...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
1,84217ca10afac622d53885b59101d512721a4ac74cae92...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
2,957e860e09bde8e26a45cd986dab1fe18289d290638bb9...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
3,b2d7bb7bb840556f3d45b0566d61552a0f8933ef4edc84...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
4,a2127fc77f43ce5fe5f8f3b40a826041f92aea04688b36...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
...,...,...,...,...,...
10274,c2b5cdf38b24a463a9083f6c4f5f9f14d3ebeb500422a7...,"{'rustc': '1.43.1 (8d69840ab 2020-05-04)', 'wa...",Rust,rustc,rustc 1.43.1 (
10275,60b966301d55d81fe43c3fcaaad97f63c0bbc4f5698c25...,"{'rustc': '1.39.0 (4560ea788 2019-11-04)', 'wa...",Rust,rustc,rustc 1.39.0 (
10276,e6b183e40f2671dc0f87e6d85070077e9bea18e7ad50cd...,"{'rustc': '1.37.0 (eae3437df 2019-08-13)', 'wa...",Rust,rustc,rustc 1.37.0 (
10277,87c0eaf67a8c19f2aa3ad9ed64813a900fb8c9ec1ef356...,{'rustc': '1.44.0-nightly (38114ff16 2020-03-2...,Rust,rustc,rustc 1.44.0-n


,language,wasm,producer,tool,tool_version
0,C99,181,173,173,173
1,C_plus_plus,6,6,6,6
2,C_plus_plus_14,41,41,41,41
3,Go,33,33,33,33
4,Rust,1328,1328,1328,1328


In [129]:
# Now do some parsing of the binaries and gather some structural information ?
# Lets start with a simple profiling gathering
import subprocess
import os
import re

def nodize(t, LN):
    return [x for x in t.split(' ') if x]

def parse_s(start, content):
    # read until  
    buff = ''
    nodes = []
    operators = []

    i = start
    while i != len(content):
        c = content[i]
        if c == '"':
            #escape
            i = i + 1
            sc = content[i]
            escape = ""

            while sc != '"' and i < len(content) - 1:
                escape += sc
                i = i + 1
                sc = content[i]

            i = i + 1
            if escape:
                #print(escape)
                nodes += nodize(escape, (-1, -1))
        elif c == '(':
            nnodes, newpos = parse_s(i + 1, content)
            i = newpos 
            if nnodes:
                # set previous buffer as a node

                if buff:
                    bh = buff
                    if "module" in bh:
                        bh = "module"

                    if bh.strip():
                        nodes += nodize(bh.strip(), (-1, -1))
                buff = ''
                nodes += [nnodes]
        elif c == ')':
            if buff:
                bh = buff
                if "module" in bh:
                    bh = "module"
                if bh.strip():
                    nodes += nodize(bh.strip(), (-1, -1))
            return nodes, i + 1
        else:
            buff += c
            i += 1
        
    return nodes[0], i

def pprint(tree, indent=0):
    operator, children = tree[0], tree[1:]

    tokens = [operator]
    #print("\t"*indent, operator)
    if type(children) == list:
        for ch in children:
            tokens += pprint(ch, indent + 1)
    return tokens
        
def clean_module(content):
    # Remove label annotations
    content, _ = re.subn(r'label (.*?)\n', '\n', content)
    # Remove inner comments for breaks ;@;;
    content, _ = re.subn(r';(.*?);', '', content)


    # Remove naming
    content, _ = re.subn(r'param ([0-9a-zA-Z\$\._-]+) i32', 'param_i32', content)
    content, _ = re.subn(r'param ([0-9a-zA-Z\$\._-]+) i64', 'param_i64', content)
    content, _ = re.subn(r'param ([0-9a-zA-Z\$\._-]+) f32', 'param_f32', content)
    content, _ = re.subn(r'param ([0-9a-zA-Z\$\._-]+) f64', 'param_f64', content)


    content, _ = re.subn(r'local ([0-9a-zA-Z\$\._-]+) i32', 'local_i32', content)
    content, _ = re.subn(r'local ([0-9a-zA-Z\$\._-]+) i64', 'local_i64', content)
    content, _ = re.subn(r'local ([0-9a-zA-Z\$\._-]+) f32', 'local_f32', content)
    content, _ = re.subn(r'local ([0-9a-zA-Z\$\._-]+) f64', 'local_f64', content)

    content, _ = re.subn(r'[ ]+i32', '', content)
    content, _ = re.subn(r'[ ]+f32', '', content)
    content, _ = re.subn(r'[ ]+i64', '', content)
    content, _ = re.subn(r'[ ]+f64', '', content)
    
    
    content, _ = re.subn(r'i32\.const \d+', 'i32.const', content)
    content, _ = re.subn(r'f32\.const \d+', 'f32.const', content)
    content, _ = re.subn(r'i64\.const \d+', 'i64.const', content)
    content, _ = re.subn(r'f64\.const \d+', 'f64.const', content)
    
    # Rename function names

    def replace(tk):
        #sys.stderr.write(f"{tk}")
        return "func "

    content, o = re.subn(r'func \$(.*?) ', replace, content)

    # Parse signature


    # replace call $x by call <>
    content, _ = re.subn(r'call ([0-9a-zA-Z\$\._]+)', 'call', content)
    # $env.abortOnCannotGrowMemory

    # replace local.get $x by local.get <> and local.set $x by local.set <>
    content, _ = re.subn(r'local.get ([0-9a-zA-Z\$\._-]+)', 'local.get', content)
    content, _ = re.subn(r'local.set ([0-9a-zA-Z\$\._-]+)', 'local.set', content)
    content, _ = re.subn(r'local.tee ([0-9a-zA-Z\$\._-]+)', 'local.tee', content)

    # replace global.get $x by global.get <> and global.set $x by global.set <>
    content, _ = re.subn(r'global.get ([0-9a-zA-Z\$\._-]+)', 'global.get', content)
    content, _ = re.subn(r'global.set ([0-9a-zA-Z\$\._-]+)', 'global.set', content)

    return content

def get_profiling(wasm, check_binary_in="../RQ1/all-binaries-metadata/all"):
    folded = subprocess.check_output(
        [
            os.environ.get("WASM2WAT"),
            "-f",
            f"{check_binary_in}/{wasm}"
        ]
    )
    folded = folded.decode()
    # clean a little bit
    folded = clean_module(folded)
    
    #print(folded)
    # s-expression parsing here
    nodes, _ = parse_s(0, folded)
    tokens = pprint(nodes)
    
    d = {}
    for t in tokens:
        if len(t) > 1:
            if t not in d:
                d[t] = [0]
            d[t][0] +=1
        
    return pd.DataFrame(d)
    


In [130]:
d1 = get_profiling("0aa5df2feb9f301c0d53be8e380b971499cc8c8e737966c515e5ae0e1c40f4a0.wasm")
display(d)

,wasm,producer,language,tool,tool_version
0,c8026b77b08a48698d98d8f08ad608e7527833b5fe3cf1...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
1,84217ca10afac622d53885b59101d512721a4ac74cae92...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
2,957e860e09bde8e26a45cd986dab1fe18289d290638bb9...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
3,b2d7bb7bb840556f3d45b0566d61552a0f8933ef4edc84...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
4,a2127fc77f43ce5fe5f8f3b40a826041f92aea04688b36...,{'clang': '10.0.0 (https://github.com/llvm/llv...,C_plus_plus_14,clang,clang 10.0.0
...,...,...,...,...,...
10274,c2b5cdf38b24a463a9083f6c4f5f9f14d3ebeb500422a7...,"{'rustc': '1.43.1 (8d69840ab 2020-05-04)', 'wa...",Rust,rustc,rustc 1.43.1 (
10275,60b966301d55d81fe43c3fcaaad97f63c0bbc4f5698c25...,"{'rustc': '1.39.0 (4560ea788 2019-11-04)', 'wa...",Rust,rustc,rustc 1.39.0 (
10276,e6b183e40f2671dc0f87e6d85070077e9bea18e7ad50cd...,"{'rustc': '1.37.0 (eae3437df 2019-08-13)', 'wa...",Rust,rustc,rustc 1.37.0 (
10277,87c0eaf67a8c19f2aa3ad9ed64813a900fb8c9ec1ef356...,{'rustc': '1.44.0-nightly (38114ff16 2020-03-2...,Rust,rustc,rustc 1.44.0-n


In [ ]:
# read all wasms as vectors
dall = pd.DataFrame()

## Producer consumer for a faster processing
import threading
import time

ll = threading.Lock()
ll2 = threading.Lock()

worklist = []
results = []
processed = 0

for i, w in enumerate(d['wasm'].values):
    worklist.append(w)

def produce(i):
    global results
    global worklist
    while True:
        with ll:
            if len(worklist) == 0:
                break
            curr = worklist.pop()
        print(f"worker {i} taking {curr}")
        
        start = time.time()
        d1 = get_profiling(f"{curr}.wasm")
        end = time.time()
        print("Result in ", end-start, "seconds")
        with ll2:
            results.append(d1)

def consume(expected):
    global processed
    global results
    global dall
    while True:
        time.sleep(10)
        if processed == expected:
            break
        
        with ll2:
            if len(results) > 0:
                chunk = results
                results = []
        for r in chunk:
            dall = pd.concat([dall, r], axis = 0)
            processed += 1
        dall.to_csv("dall.csv")

        print("Processed", processed)
            
producers = []
expected = len(worklist)
for i in range(16): #16 producers:
    th = threading.Thread(target=produce, args=(i,))
    th.start()
    producers.append(th)

# consumer
consumer = threading.Thread(target=consume, args=(expected,))
consumer.start()

for t in producers:
    t.join()
    
consumer.join()
    
display(dall)

worker 0 taking 12dd9730d0afa7e8e9d838f007327afe9fab2bed32778478d9acb554ea1c15dfworker 1 taking 87c0eaf67a8c19f2aa3ad9ed64813a900fb8c9ec1ef356adde8f9b83c561b238

worker 2 taking e6b183e40f2671dc0f87e6d85070077e9bea18e7ad50cd7e7cd31e2a9ade937bworker 3 taking 60b966301d55d81fe43c3fcaaad97f63c0bbc4f5698c25814a52793b20251fe4
worker 4 taking c2b5cdf38b24a463a9083f6c4f5f9f14d3ebeb500422a792100b4f10a8fe4bb8
worker 5 taking 2039f1d6d8d7f2eadd9e358d18fcf4f991816f75e4859032321eb5cbf6647398worker 6 taking 012870d396171e31cbaa4c4c9bfdec49265be087cd489384f212515272f42893

worker 7 taking ca7bc98b55af1a0f6cacfa2cb050ab3d4671407a2a11cd23db075ec5ee701ffbworker 8 taking ace8d55973d095f2e9a48ddf0547e2018ba399e0781566284c01fdc4f85bc933
worker 9 taking 59acf7be5b5b5d4ff9ef2b1bf0adb44dcac6946efb235980e4524f149f48733f


worker 10 taking f5a1826e501f75e5c16baa063015bb132e986774b21444b15c1139a11eeef6c6worker 11 taking 6ed81d64dc12d0b0c8d0301db7114eb5f5521476306621968568fdeacacfe450

worker 12 taking cb2829e85

Exception in thread Exception in thread Exception in thread Exception in thread Thread-200:
Traceback (most recent call last):
  File "/Users/javierca/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Thread-178Thread-75:
Traceback (most recent call last):
  File "/Users/javierca/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Thread-203:
:
Exception in thread Thread-172:
Traceback (most recent call last):
  File "/Users/javierca/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread     Thread-197Traceback (most recent call last):
  File "/Users/javierca/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread :
Exception in thread Exception in thread Traceback (most recent call last):
Thread-208:
    Exception in thread Traceback (most recent call last):
Thread-202Traceback (most recent call last):
:
Traceback (most recent call last):
Exception in thread self.run(